<a href="https://colab.research.google.com/github/MoronSlayer/Deep-Learning-Projects/blob/learner/rapids_comparison_v0.1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Environment Sanity Check #

Click the _Runtime_ dropdown at the top of the page, then _Change Runtime Type_ and confirm the instance type is _GPU_.

Check the output of `!nvidia-smi` to make sure you've been allocated a Tesla T4, P4, or P100.

In [1]:
!nvidia-smi

Fri Sep 30 00:38:05 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

#Setup:
Set up script installs
1. Updates gcc in Colab
1. Installs Conda
1. Install RAPIDS' current stable version of its libraries, as well as some external libraries including:
  1. cuDF
  1. cuML
  1. cuGraph
  1. cuSpatial
  1. cuSignal
  1. BlazingSQL
  1. xgboost
1. Copy RAPIDS .so files into current working directory, a neccessary workaround for RAPIDS+Colab integration.


In [3]:
!pip install pynvml

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 46 kB 3.8 MB/s 


In [2]:
# This get the RAPIDS-Colab install files and test check your GPU.  Run this and the next cell only.
# Please read the output of this cell.  If your Colab Instance is not RAPIDS compatible, it will warn you and give you remediation steps.
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/env-check.py

Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 300, done.
remote: Counting objects: 100% (129/129), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 300 (delta 74), reused 99 (delta 55), pack-reused 171
Receiving objects: 100% (300/300), 87.58 KiB | 12.51 MiB/s, done.
Resolving deltas: 100% (136/136), done.
Traceback (most recent call last):
  File "rapidsai-csp-utils/colab/env-check.py", line 1, in <module>
    import pynvml
ModuleNotFoundError: No module named 'pynvml'


In [ ]:
# This will update the Colab environment and restart the kernel.  Don't run the next cell until you see the session crash.
!bash rapidsai-csp-utils/colab/update_gcc.sh
import os
os._exit(00)

Updating your Colab environment.  This will restart your kernel.  Don't Panic!
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:2 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [913 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:

In [1]:
# This will install CondaColab.  This will restart your kernel one last time.  Run this cell by itself and only run the next cell once you see the session crash.
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:55
🔁 Restarting kernel...


In [1]:
# you can now run the rest of the cells as normal
import condacolab
condacolab.check()

✨🍰✨ Everything looks OK!


In [ ]:
# Installing RAPIDS is now 'python rapidsai-csp-utils/colab/install_rapids.py <release> <packages>'
# The <release> options are 'stable' and 'nightly'.  Leaving it blank or adding any other words will default to stable.
!python rapidsai-csp-utils/colab/install_rapids.py stable
import os
os.environ['NUMBAPRO_NVVM'] = '/usr/local/cuda/nvvm/lib64/libnvvm.so'
os.environ['NUMBAPRO_LIBDEVICE'] = '/usr/local/cuda/nvvm/libdevice/'
os.environ['CONDA_PREFIX'] = '/usr/local'

# RAPIDS is now installed on Colab.  You can copy your code into the cells below.  Enjoy!

In [5]:
import sklearn


# KNN using RAPIDS 
> Link to git: https://github.com/rapidsai/cuml/blob/branch-0.15/notebooks/nearest_neighbors_demo.ipynb

In [10]:
import cudf
import numpy as np
from cuml.datasets import make_blobs
from cuml.neighbors import NearestNeighbors as cuNearestNeighbors
from sklearn.neighbors import NearestNeighbors as skNearestNeighbors

### Defining parameters

In [53]:
x = np.random.rand(40000, 20)
y = np.random.randint(2, size=x.shape[0])

### Genrating data

In [54]:
%%time
# device_data, _ = make_blobs(n_samples=n_samples,
#                             n_features=n_features,
#                             centers=5,
#                             random_state=random_state)

device_data = cudf.DataFrame(x)

CPU times: user 7.55 ms, sys: 34 µs, total: 7.58 ms
Wall time: 7.61 ms


In [55]:
device_data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.118814,0.555621,0.388898,0.914401,0.391137,0.384834,0.914585,0.110585,0.161303,0.768701,0.816192,0.008138,0.441253,0.862925,0.153104,0.437370,0.092414,0.726171,0.396303,0.298044
1,0.629875,0.167532,0.829353,0.824272,0.767944,0.367768,0.899638,0.260475,0.198558,0.485597,0.552055,0.602750,0.575620,0.670358,0.641737,0.190898,0.157720,0.141883,0.191742,0.512345
2,0.988803,0.242781,0.713947,0.653577,0.607836,0.053113,0.418582,0.183786,0.728343,0.096052,0.131121,0.207549,0.114246,0.238527,0.214719,0.077386,0.356040,0.640675,0.364558,0.387095
3,0.053816,0.728705,0.285982,0.368237,0.179927,0.434823,0.563886,0.392326,0.038457,0.432901,0.222240,0.593476,0.986208,0.841993,0.526098,0.499233,0.267351,0.820124,0.900129,0.149745
4,0.890731,0.111660,0.026922,0.612394,0.145427,0.145583,0.445187,0.539904,0.760262,0.027796,0.619639,0.447905,0.945907,0.416186,0.671774,0.320414,0.992598,0.297584,0.319220,0.614795
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,0.420470,0.208267,0.861503,0.495732,0.459839,0.071873,0.487319,0.805199,0.667725,0.269683,0.629181,0.283420,0.034452,0.766648,0.348151,0.958922,0.078305,0.589331,0.564164,0.349936
39996,0.087604,0.821644,0.662202,0.132014,0.980025,0.894555,0.973740,0.613325,0.630793,0.270247,0.539228,0.949874,0.339162,0.850452,0.354018,0.553911,0.680401,0.634413,0.197455,0.781293
39997,0.306018,0.505627,0.108801,0.195087,0.421064,0.591568,0.654503,0.606096,0.546817,0.529873,0.745903,0.792820,0.422975,0.054922,0.043847,0.056648,0.882650,0.887953,0.337695,0.438711
39998,0.985133,0.692225,0.016554,0.510649,0.305933,0.794617,0.978207,0.905461,0.383053,0.683139,0.598369,0.187960,0.504513,0.060362,0.607331,0.290345,0.490356,0.511055,0.799381,0.618362


cuML model

In [51]:
%%time
knn_cuml = cuNearestNeighbors()
knn_cuml.fit(x)

CPU times: user 1.13 s, sys: 21 ms, total: 1.15 s
Wall time: 1.18 s


NearestNeighbors()

In [52]:
%%time
D_cuml, I_cuml = knn_cuml.kneighbors(x, n_neighbors=5)

CPU times: user 568 ms, sys: 179 ms, total: 748 ms
Wall time: 748 ms


In [29]:
I_cuml.shape

(100000, 5)

In [9]:
# Copy dataset from GPU memory to host memory.
# This is done to later compare CPU and GPU results.
host_data = device_data.to_pandas()

### Scikit-learn model

In [43]:
%%time
knn_sk = skNearestNeighbors(n_neighbors=5, algorithm="auto",
                            n_jobs=-1)
knn_sk.fit(x)
pred = knn_sk.kneighbors(x)

CPU times: user 1min 52s, sys: 2.69 s, total: 1min 55s
Wall time: 1min 24s


In [45]:
pred[0:5]

(array([[0.        , 0.74423908, 0.85513171, 0.94504838, 0.98477357],
        [0.        , 0.80424923, 0.84263274, 0.94143823, 0.94244218],
        [0.        , 0.85789897, 0.88378544, 0.89175769, 0.95642566],
        ...,
        [0.        , 0.93607085, 0.95921981, 0.97759025, 0.97839259],
        [0.        , 0.77815515, 0.83239857, 0.88536085, 0.88648052],
        [0.        , 0.67035888, 0.75633615, 0.88575261, 0.9032486 ]]),
 array([[    0, 20577, 27526,  5554, 18407],
        [    1, 28767,  2579, 39661, 28893],
        [    2, 27900, 36422,   959, 35347],
        ...,
        [39997, 15192, 12107, 12421, 19403],
        [39998, 17223, 17505, 24006, 36736],
        [39999, 19326, 24181, 17896,  3351]]))

In [49]:
I_cuml[0:5]

array([[    0, 20577, 27526,  5554, 18407],
       [    1, 28767,  2579, 39661, 28893],
       [    2, 27900, 36422,   959, 35347],
       [    3, 33189, 13232, 34266, 34164],
       [    4, 15766, 14364, 33310, 34866]])

In [11]:
%%time
D_sk, I_sk = knn_sk.kneighbors(host_data[:n_query], n_neighbors)

CPU times: user 56.8 s, sys: 8.26 s, total: 1min 5s
Wall time: 47.2 s




---



---



---



In [12]:
from sklearn import datasets
X, y  = datasets.make_classification(n_samples=40000)

In [13]:
X = X.astype(np.float32)
y = y.astype(np.float32)

In [14]:
def train_data(model, X=X, y=y):
    clf = model
    clf.fit(X, y)

In [15]:
from sklearn.svm import SVC 
from cuml.svm import SVC as SVC_gpu

clf_svc = SVC(kernel='poly', degree=2, gamma='auto', C=1)
sklearn_time_svc = %timeit -o train_data(clf_svc)

clf_svc = SVC_gpu(kernel='poly', degree=2, gamma='auto', C=1)
cuml_time_svc = %timeit -o train_data(clf_svc)

print(f"""Average time of sklearn's {clf_svc.__class__.__name__}""", sklearn_time_svc.average, 's')
print(f"""Average time of cuml's {clf_svc.__class__.__name__}""", cuml_time_svc.average, 's')

print('Ratio between sklearn and cuml is', sklearn_time_svc.average/cuml_time_svc.average)

1min 18s ± 3.58 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
2.48 s ± 12.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Average time of sklearn's SVC 78.65281007214298 s
Average time of cuml's SVC 2.4763528024286643 s
Ratio between sklearn and cuml is 31.761552713734826


In [16]:
!pip install cutecharts

import cutecharts.charts as ctc 

def plot(sklearn_time, cuml_time):

    chart = ctc.Bar('Sklearn vs cuml')
    chart.set_options(
        labels=['sklearn', 'cuml'],
        x_label='library',
        y_label='time (s)',
        )

    chart.add_series('time', data=[round(sklearn_time.average,2), round(cuml_time.average,2)])
    return chart

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [20]:
plot(sklearn_time_svc, cuml_time_svc).render_notebook()

In [21]:
from cuml.neighbors import (KNeighborsClassifier as KNeighborsClassifier_gpu,
                            KNeighborsRegressor as KNeighborsRegressor_gpu)
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor


In [25]:
clf_nn = KNeighborsClassifier(n_neighbors=10)
sklearn_time_nn = %timeit -o train_data(clf_nn)


3.22 ms ± 31.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [26]:
clf_nn = KNeighborsClassifier_gpu(n_neighbors=10)
cuml_time_nn = %timeit -o train_data(clf_nn)

2.47 ms ± 25.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [27]:
print(f"""Average time of sklearn's {clf_nn.__class__.__name__}""", sklearn_time_nn.average, 's')
print(f"""Average time of cuml's {clf_nn.__class__.__name__}""", cuml_time_nn.average, 's')

print('Ratio between sklearn and cuml is', sklearn_time_nn.average/cuml_time_nn.average)

Average time of sklearn's KNeighborsClassifier 0.003222532555714679 s
Average time of cuml's KNeighborsClassifier 0.002467951844285251 s
Ratio between sklearn and cuml is 1.3057517970525732


In [28]:
plot(sklearn_time_nn, cuml_time_nn).render_notebook()

# Next Steps #

For an overview of how you can access and work with your own datasets in Colab, check out [this guide](https://towardsdatascience.com/3-ways-to-load-csv-files-into-colab-7c14fcbdcb92).

For more RAPIDS examples, check out our RAPIDS notebooks repos:
1. https://github.com/rapidsai/notebooks
2. https://github.com/rapidsai/notebooks-contrib